In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests

import pandas as pd
import time
import os

In [4]:
s = Service("C:/Users/edmund/GitLocal/SitShop/PythonData/chromedriver.exe")
driver = webdriver.Chrome(service = s)

In [7]:
def review_scraper(username, driver=driver):
    driver.get(f"https://www.carousell.sg/u/{username}/reviews/")

    while True:
        try:
            load_more_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/main/div[1]/div[1]/div[3]/div[2]/div/button")
            load_more_button.click()
            time.sleep(2)
        except:
            break

    customer_cards = driver.find_elements(By.CSS_SELECTOR, "li[class^='D_WK']")
    
    customer_list = []

    counter = 0 
    for customer in customer_cards:
        if counter == 0:
            print("scraping starts!")
        elif counter%50 == 0:
            print(f"scraped {counter} customers!")
        
        customer_dict = {}
    
        customer_dict['reviewee'] = username
        
        top = customer.find_element(By.CSS_SELECTOR, "div[class^='D_OH']")
        
        reviewer = top.find_elements(By.CSS_SELECTOR, "span")[0]
        reviewer = reviewer.text
        customer_dict['reviewer'] = reviewer
        
        identity = top.find_elements(By.CSS_SELECTOR, "span")[1]
        identity = identity.text.split(" ")[-1]
        customer_dict['reviewer_identity'] = identity
    
        date = top.find_elements(By.CSS_SELECTOR, "span")[2]
        date = date.text
        customer_dict['review_date'] = date
    
        star = customer.find_element(By.CSS_SELECTOR, "div[class^='D_Hj']")
        star = star.get_attribute('aria-label').split(" ")[0]
        star = float(star)
        customer_dict['review_star'] = star

        review = customer.find_element(By.CSS_SELECTOR, "p[data-testid='feedback-review']")
        review = review.text
        customer_dict["review"] = review
    
        try:
            tag_list = customer.find_element(By.CSS_SELECTOR, "div[class^='D_OR']")
            tag_list = tag_list.find_elements(By.CSS_SELECTOR, "div[class^='D_OS']")
            for tag in tag_list:
                tag = tag.text
                customer_dict[tag] = 1
                
        except:
            pass
    
        customer_list.append(customer_dict)
        counter += 1

    return customer_list

In [6]:
data_row = 0

while data_row < 100000:
    if data_row == 0:
        customer_list = review_scraper('jeankini')
        customer_df = pd.DataFrame(customer_list)
        customer_df = customer_df.fillna(0)
        
        data_row = len(customer_df)
        print(data_row)
        display(customer_df)
        
    else:
        for seller in customer_df[customer_df['reviewer_identity'] == 'seller']['reviewer']:
            if seller not in set(customer_df['reviewee']):
                new_customer_list = review_scraper(seller)
                customer_list = customer_list + new_customer_list
                customer_df = pd.DataFrame(customer_list)
                customer_df = customer_df.fillna(0)

                data_row = len(customer_df)
                print(data_row)
                display(customer_df.tail(3))

scraping starts!
scraped 50 customers!
90


,reviewee,reviewer,reviewer_identity,review_date,review_star,review,Amazing chat,Goes the extra mile,Knows their stuff,Unique listings
0,jeankini,summersunflower,buyer,3 days ago,5.0,"Great seller, hassle free deal and mail out th...",1.0,0.0,0.0,0.0
1,jeankini,goodie2muchshoes,buyer,5 days ago,5.0,"lovely seller, thanks!",1.0,0.0,0.0,0.0
2,jeankini,muscatbuns,buyer,5 days ago,5.0,"Item exactly as per description, mailed out an...",0.0,1.0,0.0,0.0
3,jeankini,sallysazzy,buyer,10 days ago,5.0,"Nice seller, easy to deal. Appreciate that she...",0.0,1.0,0.0,0.0
4,jeankini,jjauw,buyer,20 days ago,5.0,"From a mother to another, thank you for the wo...",1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
85,jeankini,blackgreenwhite,seller,8 years ago,5.0,A sincere and decisive buyer! Thank you for th...,0.0,0.0,0.0,0.0
86,jeankini,taokaenoi,seller,8 years ago,5.0,Prompt payment. Sincere buyer.,0.0,0.0,0.0,0.0
87,jeankini,juswink,seller,8 years ago,5.0,Very prompt payment. No fuss. Thanks.,0.0,0.0,0.0,0.0
88,jeankini,dollymehmeh,buyer,8 years ago,5.0,Prompt delivery! Thank you!,0.0,0.0,0.0,0.0


scraping starts!
scraped 50 customers!
scraped 100 customers!
203


,reviewee,reviewer,reviewer_identity,review_date,review_star,review,Amazing chat,Goes the extra mile,Knows their stuff,Unique listings,Easygoing buyer,Fast and decisive
200,lifenotsimple.poh,ma.br,buyer,6 years ago,5.0,"patient and understanding seller, definitely r...",0.0,0.0,0.0,0.0,0.0,0.0
201,lifenotsimple.poh,joyousworld,buyer,6 years ago,5.0,Reliable seller. Punctual and friendly.,0.0,0.0,0.0,0.0,0.0,0.0
202,lifenotsimple.poh,rainbowsonme,buyer,6 years ago,5.0,Punctual! Speedy replies. Great seller to deal...,0.0,0.0,0.0,0.0,0.0,0.0


scraping starts!
scraped 50 customers!
scraped 100 customers!
scraped 150 customers!
scraped 200 customers!
407


,reviewee,reviewer,reviewer_identity,review_date,review_star,review,Amazing chat,Goes the extra mile,Knows their stuff,Unique listings,Easygoing buyer,Fast and decisive
404,pinky_33,cinnamonnn,seller,8 years ago,5.0,"Buyer was decisive, friendly and prompt with p...",0.0,0.0,0.0,0.0,0.0,0.0
405,pinky_33,karen.n,seller,8 years ago,5.0,It's been a smooth and fast transaction. No fu...,0.0,0.0,0.0,0.0,0.0,0.0
406,pinky_33,parisian,buyer,8 years ago,5.0,Great seller! Fast and efficient!,0.0,0.0,0.0,0.0,0.0,0.0


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"p[class='D_pw M_jT D_ov M_jO D_px M_jU D_pA M_jY D_pE M_kb D_pH M_kf D_pJ M_kh D_pM']"}
  (Session info: chrome=117.0.5938.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6CB8C52A2+57122]
	(No symbol) [0x00007FF6CB83EA92]
	(No symbol) [0x00007FF6CB70E3AB]
	(No symbol) [0x00007FF6CB747D3E]
	(No symbol) [0x00007FF6CB747E2C]
	(No symbol) [0x00007FF6CB780B67]
	(No symbol) [0x00007FF6CB76701F]
	(No symbol) [0x00007FF6CB77EB82]
	(No symbol) [0x00007FF6CB766DB3]
	(No symbol) [0x00007FF6CB73D2B1]
	(No symbol) [0x00007FF6CB73E494]
	GetHandleVerifier [0x00007FF6CBB6EF82+2849794]
	GetHandleVerifier [0x00007FF6CBBC1D24+3189156]
	GetHandleVerifier [0x00007FF6CBBBACAF+3160367]
	GetHandleVerifier [0x00007FF6CB956D06+653702]
	(No symbol) [0x00007FF6C

In [9]:
customer_df

,reviewee,reviewer,reviewer_identity,review_date,review_star,review,Amazing chat,Goes the extra mile,Knows their stuff,Unique listings,Easygoing buyer,Fast and decisive
0,jeankini,summersunflower,buyer,3 days ago,5.0,"Great seller, hassle free deal and mail out th...",1.0,0.0,0.0,0.0,0.0,0.0
1,jeankini,goodie2muchshoes,buyer,5 days ago,5.0,"lovely seller, thanks!",1.0,0.0,0.0,0.0,0.0,0.0
2,jeankini,muscatbuns,buyer,5 days ago,5.0,"Item exactly as per description, mailed out an...",0.0,1.0,0.0,0.0,0.0,0.0
3,jeankini,sallysazzy,buyer,10 days ago,5.0,"Nice seller, easy to deal. Appreciate that she...",0.0,1.0,0.0,0.0,0.0,0.0
4,jeankini,jjauw,buyer,20 days ago,5.0,"From a mother to another, thank you for the wo...",1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
402,pinky_33,elleon,buyer,8 years ago,5.0,Great seller! Super prompt with replies and co...,0.0,0.0,0.0,0.0,0.0,0.0
403,pinky_33,3ladies3mens,seller,8 years ago,5.0,Great experience with her..easy breezy:),0.0,0.0,0.0,0.0,0.0,0.0
404,pinky_33,cinnamonnn,seller,8 years ago,5.0,"Buyer was decisive, friendly and prompt with p...",0.0,0.0,0.0,0.0,0.0,0.0
405,pinky_33,karen.n,seller,8 years ago,5.0,It's been a smooth and fast transaction. No fu...,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
!pip install openpyxl

In [16]:
DIR = f'asset/2023/10/03/'
FILENAME = f'data_03.csv'

os.makedirs(DIR, exist_ok=True)

with open(DIR + FILENAME, 'wb') as file:
    customer_df.to_csv(file, encoding='utf-8')

In [154]:
_set = set(customer_df['reviewee'])

customer_df[customer_df['reviewer'].isin(_set)]

,reviewee,reviewer,reviewer_identity,review_date,review_star,Amazing chat,review,Goes the extra mile,Knows their stuff,Unique listings,Fast and decisive,Easygoing buyer
25,jeankini,lifenotsimple.poh,seller,1 year ago,5.0,NaN,Fast deal. Prompt payment. Quick reply. Thank ...,NaN,NaN,NaN,NaN,NaN
43,jeankini,pinky_33,seller,2 years ago,5.0,NaN,fast and decisive buyer. 🤗👍,NaN,NaN,NaN,NaN,NaN
96,lifenotsimple.poh,jeankini,buyer,1 year ago,5.0,NaN,Seller wrap the item very well!,1.0,NaN,NaN,NaN,NaN
229,pinky_33,jeankini,buyer,2 years ago,5.0,NaN,Very nice seller and she packs the item well a...,NaN,NaN,NaN,NaN,NaN


In [113]:
customer_df = pd.DataFrame(customer_list)
display(customer_df.info())
display(customer_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   reviewee             90 non-null     object 
 1   reviewer             90 non-null     object 
 2   reviewer_identity    90 non-null     object 
 3   review_date          90 non-null     object 
 4   review_star          90 non-null     float64
 5   Amazing chat         16 non-null     float64
 6   review               90 non-null     object 
 7   Goes the extra mile  13 non-null     float64
 8   Knows their stuff    4 non-null      float64
 9   Unique listings      4 non-null      float64
dtypes: float64(5), object(5)
memory usage: 7.2+ KB


None

,reviewee,reviewer,reviewer_identity,review_date,review_star,Amazing chat,review,Goes the extra mile,Knows their stuff,Unique listings
0,jeankini,summersunflower,buyer,3 days ago,5.0,1.0,"Great seller, hassle free deal and mail out th...",NaN,NaN,NaN
1,jeankini,goodie2muchshoes,buyer,5 days ago,5.0,1.0,"lovely seller, thanks!",NaN,NaN,NaN
2,jeankini,muscatbuns,buyer,5 days ago,5.0,NaN,"Item exactly as per description, mailed out an...",1.0,NaN,NaN
3,jeankini,sallysazzy,buyer,10 days ago,5.0,NaN,"Nice seller, easy to deal. Appreciate that she...",1.0,NaN,NaN
4,jeankini,jjauw,buyer,20 days ago,5.0,1.0,"From a mother to another, thank you for the wo...",1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
85,jeankini,blackgreenwhite,seller,8 years ago,5.0,NaN,A sincere and decisive buyer! Thank you for th...,NaN,NaN,NaN
86,jeankini,taokaenoi,seller,8 years ago,5.0,NaN,Prompt payment. Sincere buyer.,NaN,NaN,NaN
87,jeankini,juswink,seller,8 years ago,5.0,NaN,Very prompt payment. No fuss. Thanks.,NaN,NaN,NaN
88,jeankini,dollymehmeh,buyer,8 years ago,5.0,NaN,Prompt delivery! Thank you!,NaN,NaN,NaN


In [116]:
for seller in customer_df[customer_df['reviewer_identity'] == 'seller']['reviewer']:
    print(seller)

lifenotsimple.poh
pinky_33
pearlygates
affordable.deals
bestbestdeals88
cookiemoonster
kidzlove
cloud9n3r
onestopfitsall
habswags
yvonne1486
jessie.kou
cookiemoonster
ctancj
prioritytututu
lai.k.yau
thatstreetstall
chinoir
blackgreenwhite
taokaenoi
juswink


In [120]:
'jeankini' in set(customer_df['reviewee'])

True